# Analyze person image with ToolUse or not
- [Tool use API examples](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use-examples.html)

In [9]:
# is_needed_install = True
is_needed_install = False

if is_needed_install:
    # ! pip install -U boto3 langchain_aws anthropic
    ! pip install -U boto3 

In [10]:
%load_ext autoreload
%autoreload 2

from utils_bedrock_skin_gender import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Load Image

In [11]:
prepped_images = "skin_gender_img/amazon_big_size_photo.jpeg"

## 3. Run experiment

In [36]:
from textwrap import dedent

prompt = dedent(
    """
    이미지의 인물을 설명해줘
    """
)

model_id ="anthropic.claude-3-5-sonnet-20240620-v1:0"
# model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
# model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"

tool_config = {
"tools": [
    {
        "toolSpec": {
            "name": "Comments",
            "description": "이미지를 분석해서 얼굴의 특징을 뽑아내고, 얼굴색 및 성별에 맞는 중세시대 기사의 영웅적인 자화상을 만드는 프롬프트를 생성해줘",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "korean_comment_1": {
                            "type": "string",
                            "description": "이미지를 분석해서 얼굴의 특징을 뽑아내서, 중세시대 여성 기사의 영웅적인 자화상을 만드는 프롬프트를 생성해줘"
                        },
                        "korean_comment_2": {
                            "type": "string",
                            "description": "성별을 알려줘"
                        },
                        "korean_comment_3": {
                            "type": "string",
                            "description": "피부색을 알려줘"
                        },

                    },
                    "required": [
                        "korean_comment_1",
                        "korean_comment_2",
                        "korean_comment_3",                        
                    ]
                }
            }
        }
    }
]
}

# inference config
inference_config = {
    "maxTokens": 4096,
    "temperature": 0,
    "topP": 0.1,
    "stopSequences": ["\n\nHuman:"]
}

bedrock_client = boto3.client(service_name='bedrock-runtime')




### Claude 에서 이미지 전달하여, Prommt 에 반응

In [37]:
try:
    isToollUse = False
    messages = generate_text(bedrock_client, model_id, tool_config, 
                            prompt, 
                            prepped_images,
                            inference_config,
                            # tools=isToollUse,
                            tools=isToollUse,   
                            verbose=False,                         
                            )


except ClientError as err:
    messages = err.response['Error']['Message']
    print("A client error occurred: %s", message)
    print(f"A client error occured: {message}")

if isToollUse:
    parse_korean_comments_tools(messages)
else:
    parse_korean_comments(messages)


## Generating text with model %s anthropic.claude-3-5-sonnet-20240620-v1:0
ToolUse is NOT used


이 이미지는 검은 배경 앞에서 찍은 한 남성의 얼굴 사진입니다.
그는 아시아계로 보이며, 짧은 검은 머리를 하고 있습니다.
그의 표정은 따뜻하고 친근해 보이며, 부드러운 미소를 짓고 있습니다.
그는 밝은 회색 또는 흰색 상의를 입고 있는 것으로 보입니다.
전반적으로 이 사진은 전문적이면서도 친근한 느낌을 줍니다.


### Claude 에서 이미지 전달하고, ToolUse 를 이용하여 Prommt 에 반응

In [38]:
try:
    isToollUse = True

    messages = generate_text(bedrock_client, model_id, tool_config, 
                            prompt, 
                            prepped_images,
                            inference_config,
                            tools=isToollUse,   
                            verbose=True,                         
                            )


except ClientError as err:
    messages = err.response['Error']['Message']
    print("A client error occurred: %s", message)
    print(f"A client error occured: {message}")

if isToollUse:
    parse_korean_comments_tools(messages)
else:
    parse_korean_comments(messages)


## Generating text with model %s anthropic.claude-3-5-sonnet-20240620-v1:0
ToolUse is used


A client error occurred: %s Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.
A client error occured: Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


AttributeError: 'str' object has no attribute 'get'